# Decision Stump



### Learning Objectives 
- Construct a decision stump with a split on one node and use the majority vote algorithm at leaf nodes
- Become familiar with a machine learning algorithm and mindset
- Gain further familiarity coding in Python and related modules (e.g. NumPy)

<a name="data"></a>

# Part I: Data Inspection

We begin by inspecting the dataset. For this assignment, we use a modified version of the *heartFailure dataset*, which contains heart failure clinical records. The original dataset consists of 299 instances and 13 attributes (*e.g.* age, diabetes, smoking, etc.). **In this assignment, we only  handle attributes with binary values** (*e.g.* 0/1, yes/no).  The modified dataset consists of 299 instances and 5 binary attributes. More information on the heartFailure dataset can be found on this [link](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records).




In [ ]:
from typing import Tuple, List, Dict, Any

import numpy as np
import pandas as pd
import csv

from testing import TestHelper

TOY_TRAIN_FILE = './data/toy_train.csv'     # filepath of toy_train dataset
TOY_TEST_FILE = './data/toy_test.csv'       # filepath of toy_test dataset
FULL_TRAIN_FILE = './data/full_train.csv'   # filepath of full_train dataset
FULL_TEST_FILE = './data/full_test.csv'     # filepath of full_test dataset

Run the cell below to visualize the `toy_train` dataset. The first column is a pandas DataFrame index column that can be ignored.

In [ ]:
toy_train = pd.read_csv(TOY_TRAIN_FILE) # load data into a panda DataFrame
toy_test = pd.read_csv(TOY_TEST_FILE)
print(toy_train)                        # show toy_train data

   high_blood_pressure  smoking  DEATH_EVENT
0                    0        0            0
1                    1        0            0
2                    1        0            1
3                    0        1            0
4                    0        1            1
5                    0        0            1
6                    0        0            0
7                    1        1            1
8                    0        0            0
9                    0        1            0


To help you with data preprocessing, we provide you with the function `load_data` that 
takes in a filepath of the train/test dataset and output a tuple of `(X, Y, attribute_names)`, where: 
- `X` is a NumPy array with $N$ rows and $M$ columns (*i.e.* shape `(N, M)`), where $N, M$ denote the number of instances and attributes, respectively
- `Y` is a NumPy array with shape `(N, )`, where $N$ is number of instances. **Note**: there is a crucial difference between shape `(N,)` vs. `(N, 1)` in NumPy: The former represents an 1-dimension array with $N$ elements, while the latter represents a 2-D array with $N$ rows and 1 column.
- `attribute_names`: a list of string, with the index corresponds to the column index of `X` (*e.g.* `['high_blood_pressure', 'smoking']`).

In [ ]:
def load_data(filename: str) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """This function takes in the filepath of the data and outputs the tuple of 
    (X, Y, attribute_names). This reader assumes the label Y is always positioned 
    at the last column

    Parameters
    ----------
    filename: type `str`
        The filepath to the dataset

    Returns
    -------
    A tuple (X, Y, attributes_name) where
        X: type `np.ndarray`, shape (N, M)
            Numpy arrays with N rows, M columns containing the attribute values for N training instances
        Y: type `np.ndarray`, shape (N, )
            Numpy arrays with N rows containing the true labels for N training instances
        attribute_names: type `List[str]`
            Names of the attributes
    """
    X: List[str] = []
    Y: List[str] = []
    attribute_names: List[str] = []
    with open(filename, 'r') as f: 
        reader = csv.reader(f)

        # get attribute list, get rid of label header
        attribute_names = next(reader)[:-1] 

        # get X, Y values and convert them to numpy arrays
        for row in reader: 
            X.append(row[:-1])
            Y.append(row[-1])
        X = np.array(X)
        Y = np.array(Y)

    return (X, Y, attribute_names)

Load the following cell to visualize `X`, `Y`, and `attribute_names` of the `toy_train` dataset. Don't forget to load the cell containing `load_data` function above. Questions 1 - 3 then check your understanding of the data

In [ ]:
toy_X_train, toy_Y_train, toy_attributes = load_data(TOY_TRAIN_FILE)
print('Shape of X =', toy_X_train.shape)
print('X = ', toy_X_train)
print('Shape of Y = ', toy_Y_train.shape)
print('Y = ', toy_Y_train)
print('attribute_names =', toy_attributes)

Shape of X = (10, 2)
X =  [['0' '0']
 ['1' '0']
 ['1' '0']
 ['0' '1']
 ['0' '1']
 ['0' '0']
 ['0' '0']
 ['1' '1']
 ['0' '0']
 ['0' '1']]
Shape of Y =  (10,)
Y =  ['0' '0' '1' '0' '1' '1' '0' '1' '0' '0']
attribute_names = ['high_blood_pressure', 'smoking']


[Back to top](#index)

<a name="q1"></a>
### Question 1: Number of Attribute [5 pt]
What is the number of attributes in the `toy_train` dataset? Assign your answer as a Python integer to variable `ans1` below.

In [ ]:

ans1 = toy_X_train.shape[1]




### Name of Attribute 

What attribute does column index 1 of `X` in `toy_train` represent? Note that in Python, indexing starts from 0 (*i.e.* index 0 denotes the first element). Assign your answer as a Python string to variable `ans2` below.

In [ ]:

ans2 = toy_attributes[1] 


 
### Value of Attribute

What are all the possible values of the attribute in question 2, in the heartFailure dataset? Please list them in ascending/alphabetical order in a Python list and assign your answer to variable `ans3` (*e.g.* `ans3 = ['value1', 'value2']`). Each element in the list should have type string.

In [ ]:

ans3 = sorted(toy_X_train[1]) 


['0', '1']


<a name="create-data"></a>

### Create Your Own Data

In addition to the `toy` dataset, you are welcome to create your own data for debugging purposes. You can create the data directly in the code block, instead of loading it from a file. Below we show you an example of `X` array with 4 rows (instances) and 2 columns (attributes), and the associated label array `Y`.

In [ ]:
# An example of X array with shape (4, 2) and Y array with shape (4,).
# The values are all 0 as placeholder, but you can modify the value 
# and the arrays as you wish. 
X = np.array([
    ['0','0'],
    ['0','0'],
    ['0','0'],
    ['0','0']
])
Y = np.array([
    '0',
    '0',
    '0',
    '0'
])



# Part II: Implementation



<a name="q4"></a>
 Majority Vote 

To begin, implement a `majority_vote` algorithm that takes in `X, Y` NumPy arrays and outputs the desired label. If there are an equal number of labels, the tie-breaker goes to the label value that appears first in the dataset (*i.e.* if there is an equal number of `'0'` and `'1'`, and `'1'` appears first, then `majority_vote` outputs `'1'`). In addition, you are guaranteed that there is no empty dataset in our test cases.


In [ ]:
 
def majority_vote(X: np.ndarray, Y: np.ndarray) -> str:
    """This function computes the output label of the given dataset, following the 
    majority vote algorithm

    Parameters
    ----------
    X: type `np.ndarray`, shape (N, M)
        Numpy arrays with N rows, M columns containing the attribute values for N instances
    Y: type `np.ndarray`, shape (N, )
        Numpy arrays with N rows containing the true labels for N instances

    Returns the majority label
    """

    d = {}
    for i in Y:
        if i in d:
            d[i]+=1
        else:
            d[i] =1
    test_val = list(d.values())[0]
  
    for ele in d:
        if d[ele] != test_val:
            inverse = [(value, key) for key, value in d.items()]
            return (max(inverse)[1])
    return Y[0] 


In [ ]:

X = np.array([
    ['0','0'],
    ['0','0'],
    ['0','0'],
    ['0','0']
])
Y = np.array([
    '1',
    '0',
    '0',
    '0'
])

# call your function on the toy dataset
label = majority_vote(X, Y)

# print the result
print('label=', label) # expected answer is '0', type 'str'

label= 0


In [ ]:
# Example output of your majority_vote function on toy train dataset
toy_X_train, toy_Y_train, toy_attributes = load_data(TOY_TRAIN_FILE)
label = majority_vote(toy_X_train, toy_Y_train)
print(label) # expected answer is '0', type 'str'

0




<a name="q5"></a>

 Split 

Next, implement a `split` function that takes in a dataset (`X,Y` arrays), 
the attribute to split on (represented as an column index of `X`), and returns a tuple of the split datasets.
Specifically: 
- For input `X` and `Y`, you should output the tuple `(X_left, Y_left, X_right, Y_right)`, all of type NumPy arrays. 
- The left and right values of the attribute should be in alphabetical order (e.g. `'0'` on the left, `'1'` on the right). The left sub-dataset (`X_left, Y_left`) should corresponds to the left attribute value, and similarly for the right sub-dataset (`X_right`, `Y_right`).
- `X` is guaranteed to be split into 2 non-empty datasets. In other words, the values of the split attributes in `X` is always more than 1 (otherwise, we would already be in a leaf node, and there is no need to keep splitting). As a side note when implementing decision tree, you should always check for this condition before splitting.

 

In [ ]:

def split(X: np.ndarray, 
          Y: np.ndarray, 
          split_attribute: int
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """This function takes a dataset and splits it into two sub-datasets according 
    to the values of the split attribute. The left and right values of the split 
    attribute should be in alphabetical order. The left dataset should correspond 
    to the left attribute value, and similarly for the right dataset. 

    Parameters
    ----------
    X: type `np.ndarray`, shape (N, M)
        Numpy arrays with N rows, M columns containing the attribute values for N instances
    Y: type `np.ndarray`, shape (N, )
        Numpy arrays with N rows containing the true labels for N instances
    split_attribute: type `int`
        The index of the attribute to split the dataset on

    Returns the tuple of two sub-datasets, i.e. (X_left, Y_left, X_right, Y_right)
    """
    
    keys = set()
    for i in X:
        keys.add(i[split_attribute])
        
    keys =sorted(list(keys))
    X_left, Y_left, X_right, Y_right = [], [], [],[]
    for i in range(len(X)):
        if X[i][split_attribute] == keys[0]:
            X_left.append(X[i])
            Y_left.append(Y[i])
        else:
            X_right.append(X[i])
            Y_right.append(Y[i])
    return (np.array(X_left), np.array(Y_left), np.array(X_right), np.array(Y_right)) # replace this line with your return statement



Similar to `majority_vote`, you can inspect your function output by calling `split` on either your own created dataset, or load dataset from file.

In [ ]:
# Create a toy dataset for you to debug. Replace X, Y with the test cases that
# you did not pass.  You can load dataset with something like
# X, Y, attribute = load_data(TOY_TRAIN_FILE)
X = np.array([
    ['1','0'],
    ['1','0'],
    ['0','0'],
    ['0','1']
])
Y = np.array([
    '1',
    '0',
    '0',
    '0'
])
split_attribute = 0  

# call your function on the toy dataset
X_left, Y_left, X_right, Y_right = split(X, Y, split_attribute=split_attribute)
 
# print the result. You can print X_left, Y_left, X_right, Y_right as well
print(X_left)
# expected result for X_left is 
# [['0' '0']
#  ['0' '1']]

[['0' '0']
 ['0' '1']]





### Train 

Implement the `train` function for your decision stump. This function takes in train dataset `X_train, Y_train`, and the index of the split attribute `attribute_index`. The output of this function is a tuple of two strings, where the first element of the tuple is the label of the left child node, and the second element is the label of the right child node. 

.

In [ ]:

def train(X_train: np.ndarray, Y_train: np.ndarray, attribute_index: int) -> Tuple[str, str]:
    """This function takes the training dataset and a split attribute and outputs the 
    tuple of (left_label, right_label), corresponding the label on the left and right 
    leaf nodes of the decision stump

    Parameters
    ----------
    X_train: type `np.ndarray`, shape (N, M)
        Numpy arrays with N rows, M columns containing the attribute values for N training instances
    Y_train: type `np.ndarray`, shape (N, )
        Numpy arrays with N rows containing the true labels for N training instances
    attribute_index: type `int`
        The index of the attribute to split the dataset on

    Returns the tuple of labels, i.e. (left_label, right_label)
    """
    
    X_left, Y_left, X_right, Y_right = split(X_train, Y_train, split_attribute=split_attribute)
    left_label = majority_vote(X_left, Y_left)
    right_label = majority_vote(X_right, Y_right)
    return (left_label, right_label) # replace this line with your return statement



In [ ]:
# Create a toy dataset for you to debug. Replace X, Y with the test cases that
# you did not pass.  You can load dataset with something like
# X, Y, attribute = load_data(TOY_TRAIN_FILE)
X = np.array([
    ['1','0'],
    ['1','0'],
    ['0','0'],
    ['0','1']
])
Y = np.array([
    '1',
    '0',
    '0',
    '0'
])
split_attribute = 0  

# call your function on the toy dataset
left_label, right_label = train(X, Y, attribute_index=split_attribute)

# print the result
print('left label =', left_label)   # expected result is '0'
print('right label =', right_label) # expected result is '1'

left label = 0
right label = 1




### Predict

Implement the `predict` function that takes in your trained stump (output of the `train` function), an `X` array, the split `attribute_index` and predicts the labels of `X`. The output should be a 1-D NumPy array with the same number of elements as instances in `X`.


In [ ]:

def predict(left_label: str, right_label: str, X: np.ndarray, attribute_index: int) -> np.ndarray:
    """This function takes in the output of the train function (left_label, right_label), 
    the dataset without label (i.e. X), and the split attribute index, and returns the 
    label predictions for X

    Parameters
    ----------
    left_label: type `str`
        The label corresponds to the left leaf node of the decision stump
    right_label: type `str`
        The label corresponds to the right leaf node of the decision stump
    X: type `np.ndarray`, shape (N, M)
        Numpy arrays with N rows, M columns containing the attribute values for N instances
    attribute_index: type `int`
        The index of the attribute to split the dataset on

    Returns the numpy arrays with shape (N,) containing the label predictions for X
    """
    
    ans  =  []
    keys = set()
    for i in X:
        keys.add(i[split_attribute])
        
    keys =sorted(list(keys))
    for i in range(len(X)):
        if X[i][split_attribute] == keys[0]:
            ans.append(left_label)
        else:
            ans.append(right_label)
        
    return np.array(ans) 

In [ ]:
# Create a toy dataset for you to debug. Replace X, Y with the test cases that
# you did not pass. You can load dataset with something like
# X, Y, attribute = load_data(TOY_TRAIN_FILE)
X = np.array([
    ['1', '0'],
    ['1', '0'],
    ['0', '0'],
    ['0', '0']
])
(left_label, right_label) = ('1', '1')
split_attribute = 0  

# call your function on the debug dataset
Y_pred = predict(left_label, right_label, X, attribute_index=split_attribute)

# print the result
print(Y_pred) # expected result is ['1' '1' '1' '1']

['1' '1' '1' '1']



### Error Rate

Implement the `error_rate` function that takes in the true `Y` values and the `Y_pred` predictions (output of `predict` function) and computes the error rate, which is the number of incorrect instances divided by the number of total instances.

In [ ]:

def error_rate(Y: np.ndarray, Y_pred: np.ndarray) -> float:    
    """This function computes the error rate (i.e. number of incorrectly predicted
    instances divided by total number of instances)

    Parameters
    ----------
    Y: type `np.ndarray`, shape (N, )
        Numpy arrays with N rows containing the true labels for N instances
    Y_pred: type `np.ndarray`, shape (N, )
        Numpy arrays with N rows containing the predicted labels for N instances

    Returns the error rate, which is a float value between 0 and 1 
    """
    corr = len(Y)
    incorr = 0
    for i in range(len(Y)):
        if Y[i] != Y_pred[i]:
            incorr+=1
        
    return incorr/corr 

In [ ]:
# Create a toy dataset for you to debug. Replace Y, Y_pred with the test cases that
# you did not pass.  You can load dataset with something like
# X, Y, attribute = load_data(TOY_TRAIN_FILE)
Y = np.array(['1','0','0','0'])
Y_pred = np.array(['1','1','0','0'])

# call your function
rate = error_rate(Y, Y_pred)

# print the result
print(rate) # expected result is 0.25

0.25



### Putting Everything Together

In this last task, you should make use of the previous functions to implement `train_and_test` to obtain the results of your decision stump given a dataset. The function `train_and_test` takes in the filepath of the train dataset, the filepath of the test dataset, the split attribute index, and returns the dictionary containing the relevant outputs (*e.g.* train and test error rates). 

In [ ]:

def train_and_test(train_filename: str, test_filename: str, attribute_index: int) -> Dict[str, Any]:
    """This function ties the above implemented functions together. The inputs are 
    filepaths of the train and test datasets as well as the split attribute index. The
    output is a dictionary of relevant information (e.g. train and test error rates)

    Parameters
    ----------
    train_filename: type `str`
        The filepath to the training file
    test_filename: type `str`
        The filepath to the testing file
    attribute_index: type `int`
        The index of the attribute to split the dataset on

    Returns an output dictionary
    """
    X_train, Y_train, attribute_names = load_data(train_filename) 
    X_test, Y_test, _ = load_data(test_filename)

    left_label, right_label = train(X_train, Y_train, attribute_index) 
    Y_pred_train = predict(left_label, right_label, X_train, attribute_index) 
    Y_pred_test = predict(left_label, right_label, X_test, attribute_index) 

    train_error_rate = error_rate(Y_train, Y_pred_train) 
    test_error_rate = error_rate(Y_test, Y_pred_test) 

    return {
        'attribute_names' : attribute_names,
        'stump'           : (left_label, right_label),
        'train_error_rate': train_error_rate,
        'test_error_rate' : test_error_rate
    }
    



In [ ]:
train_file = FULL_TRAIN_FILE 
test_file = FULL_TEST_FILE
attribute_index = 0

#train_file = TOY_TRAIN_FILE
#test_file = TOY_TEST_FILE
#attribute_index = 0
# call your function
output_dict = train_and_test(train_file, test_file, attribute_index=attribute_index)
print(train_file)
print(test_file)
print(attribute_index)
# print the result
print('attributes: ', output_dict['attribute_names'])
print('stump: ', output_dict['stump'])  
print('train error: ',output_dict['train_error_rate']) # expected result is 0.3
print('test error: ', output_dict['test_error_rate']) # expected result is 0.34

./data/full_train.csv
./data/full_test.csv
0
attributes:  ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
stump:  ('0', '0')
train error:  0.3
test error:  0.3422818791946309
